## Coding Exercise #0703

In [ ]:
#Uncomment to force use of CPU.
#import os
#os.environ['CUDA_VISIBLE_DEVICES'] = ''

#Suppress TF warnings
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

### 1. Softmax regression (multi-class logistic regression):

In [ ]:
# import tensorflow as tf
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

#### 1.1. Read in the data:

In [ ]:
# We will use Iris data.
# 4 explanatory variables.
# 3 classes for the response variable.
data_raw = load_iris()
data_raw.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [ ]:
X = data_raw['data']
y = data_raw['target']

In [ ]:
# Check the shape.
print(X.shape)
print(y.shape)

(150, 4)
(150,)


#### 1.2. Data pre-processing:

In [ ]:
# One-Hot-Encoding.
y = np.array(pd.get_dummies(y, drop_first=False))               # drop_frist = False for one-hot-encoding.
y.shape

(150, 3)

In [ ]:
# Scaling
X = scale(X).astype('float32')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=3)
n_train_size = y_train.shape[0]

#### 1.3. Do the necessary definitions:

In [ ]:
batch_size = 100                                # Size of each (mini) batch.
n_epochs  = 30000                               # Number of epochs.
learn_rate = 0.05

In [ ]:
W = tf.Variable(tf.ones([4,3],dtype=tf.float32))                 # Initial value of the weights = 1.
b = tf.Variable(tf.ones([3],dtype=tf.float32))                   # Initial value of the bias = 1.

2024-05-17 09:12:52.250462: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [ ]:
# Model.
def model(x):
    # Not strictly necessary to apply the softmax activation. => in the end we will apply argmax() function to predict the label!
    # return tf.nn.softmax(tf.matmul(X, W) + b)
    return tf.matmul(x, W) + b

In [ ]:
# Loss = cross entropy.
def loss_fn(y_true, y_pred):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_true, y_pred))

In [ ]:
optimizer = tf.optimizers.SGD(learning_rate=learn_rate)

#### 1.4. Training and Testing:

In [ ]:
@tf.function
def train_step(X, y):
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = loss_fn(y, y_pred)
    gradients = tape.gradient(loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W, b]))

# Training.
for i in range(n_epochs):
    idx_rnd = np.random.choice(range(n_train_size), batch_size, replace=False)  # Random sampling w/o replacement for the batch indices.
    batch_X, batch_y = [X_train[idx_rnd, :], y_train[idx_rnd, :]]               # Get a batch.
    train_step(batch_X, batch_y)
    if (i + 1) % 2000 == 0:
        print("Step : {}".format(i + 1))                                        # Print the step number at every multiple of 2000.

# Testing.
correct_predictions = tf.equal(tf.argmax(y_test, axis=1), tf.argmax(model(X_test), axis=1))  # In argmax(), axis=1 means horizontal direction.
accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))                            # Recast the Boolean as float32 first. Then calculate the mean.
accuracy_value = accuracy.numpy()                                                              # Actually run the test with the test data.

Step : 2000
Step : 4000
Step : 6000
Step : 8000
Step : 10000
Step : 12000
Step : 14000
Step : 16000
Step : 18000
Step : 20000
Step : 22000
Step : 24000
Step : 26000
Step : 28000
Step : 30000


Print the testing result.

In [ ]:
print("Accuracy = {:5.3f}".format(accuracy_value))

Accuracy = 0.933
